<a href="https://colab.research.google.com/github/RohitM2997/Rohit_kaggle-/blob/main/disaster_tweets(nlp).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import pandas as pd
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [ ]:
train.isnull().sum()

,0
id,0
keyword,61
location,2533
text,0
target,0


In [ ]:
test.isnull().sum()

,0
id,0
keyword,26
location,1105
text,0


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3263 non-null   int64 
 1   keyword   3237 non-null   object
 2   location  2158 non-null   object
 3   text      3263 non-null   object
dtypes: int64(1), object(3)
memory usage: 102.1+ KB


In [ ]:
train.describe()

,id,target
count,7613.000000,7613.00000
mean,5441.934848,0.42966
std,3137.116090,0.49506
min,1.000000,0.00000
25%,2734.000000,0.00000
50%,5408.000000,0.00000
75%,8146.000000,1.00000
max,10873.000000,1.00000


In [ ]:
train.shape

(7613, 5)

In [ ]:
test.shape

(3263, 4)

In [ ]:
import matplotlib.pyplot as plt
plt.pie(train['target'].value_counts(),labels=['0','1'],autopct='%0.2f')

In [ ]:
sns.countplot(x='target',data=train)

In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(10,6))
sns.heatmap(train.isnull())
plt.show()

In [ ]:
train['location'].value_counts()

In [ ]:
train['keyword'].value_counts()

In [ ]:
train['keyword']=train['keyword'].fillna('unknown')
test['keyword']=test['keyword'].fillna('unknown')

In [ ]:
train['target'].drop_duplicates()

In [ ]:
# Text Cleaning
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'@\w+', '', text)  # Remove mentions
    text = re.sub(r'#', '', text)  # Remove hashtags but keep the word
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = text.lower()  # Convert to lowercase
    return text

# Apply cleaning
train['text'] = train['text'].apply(clean_text)
test['text']=test['text'].apply(clean_text)



In [ ]:
import nltk
nltk.download('punkt_tab')

In [ ]:
import re
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
def clean_text_remove_numbers(text):
    # Tokenize the text
    tokens = word_tokenize(text.lower())  # Convert to lowercase and tokenize

    # Remove tokens that contain numbers (using regular expressions)
    tokens = [word for word in tokens if not re.search(r'\d', word)]
    cleaned_text = " ".join(tokens)
    return cleaned_text
train['text']=train['text'].apply(clean_text_remove_numbers)
test['text']=test['text'].apply(clean_text_remove_numbers)

In [ ]:
from nltk.tokenize import word_tokenize
def word_tokenization(text):
    return nltk.word_tokenize(text)

train['text']=train['text'].apply(word_tokenization)
test['text']=test['text'].apply(word_tokenization)

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')

In [ ]:
import spacy
import nltk

from nltk import pos_tag
def get_word_vectors(text):
    """Generates word embeddings using spaCy for each token in text."""
    doc = nlp(" ".join(text))  # Joining words into a sentence for spaCy processing
    return [token.vector for token in doc]
train['text'] = train['text'].apply(pos_tag)
test['text']=test['text'].apply(pos_tag)

In [ ]:
#remove stopwards
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    return [word for word in text if word not in stop_words] # Iterate directly through the list of words

train['text'] = train['text'].apply(remove_stopwords)
test['text']=test['text'].apply(remove_stopwords)

In [ ]:
#lemma
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

# Function to apply lemmatization
def lemmatize_text(text):
    """Lemmatizes a list of words using NLTK's WordNetLemmatizer.
    """
    return [lemmatizer.lemmatize(word) for word in text] # Iterate directly through list of words

# Apply lemmatization
train['text'] = train['text'].apply(lemmatize_text)
test['text']=test['text'].apply(lemmatize_text)

In [ ]:
# prompt: apply word2vec using spacy

!pip install spacy
!python -m spacy download en_core_web_md

import spacy

# Load the pre-trained spaCy model
nlp = spacy.load("en_core_web_md")

# Function to apply Word2Vec
def apply_word2vec(text):
    # Extract only the words from the list of (word, POS tag) tuples
    text = ' '.join([word for word, pos in text])
    doc = nlp(text)
    word_vectors = [token.vector for token in doc]  # Extract word vectors
    # You can further process or analyze these vectors as needed
    # For example, calculate document vectors by averaging the word vectors
    return word_vectors

# Apply Word2Vec to the 'text' column of your DataFrame
train['word_vectors'] = train['text'].apply(apply_word2vec)
test['word_vectors'] = test['text'].apply(apply_word2vec)

In [ ]:
train_y=train['target']

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
# Transform X_train into a 2D numpy array by averaging the word vectors
X_train_new = train['word_vectors'].apply(lambda x: np.mean(x, axis=0) if len(x) > 0 else np.zeros(300))
X_train_new = np.array(list(X_train_new))

X_train, X_test, y_train, y_test = train_test_split(X_train_new, train_y, test_size=0.2, random_state=42)

In [ ]:
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score



# Initialize and train a Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)  # Adjust n_estimators as needed
rf_model.fit(X_train, y_train)  # Use the transformed data

# Make predictions on the training data
y_pred_train = rf_model.predict(X_test)

# Evaluate the accuracy on the training data
accuracy = accuracy_score(y_test, y_pred_train)
print("Train Accuracy:",accuracy)



In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Generate the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_train)

# Display the confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=rf_model.classes_)
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()


In [ ]:
# Make predictions on the test data (for submission)
X_test_new = test['word_vectors'].apply(lambda x: np.mean(x, axis=0) if len(x) > 0 else np.zeros(300))
X_test_new = np.array(list(X_test_new))
y_pred_rf = rf_model.predict(X_test_new)
y_pred_rf # Use the transformed test data

In [ ]:
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [ ]:
def evaluate_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

In [ ]:
models = {
    "Support Vector Machine": SVC(kernel='linear'),
    "Decision Tree": DecisionTreeClassifier(),
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    # "Naive Bayes": MultinomialNB()
}

In [ ]:
print("Model Performance Comparison:")

for model_name, model in models.items():
    accuracy = evaluate_model(model, X_train, y_train, X_test, y_test)
    print(f"{model_name} Accuracy: {accuracy:.2f}")